In [ ]:
# Install libraries
!pip install langchain chromadb sentence-transformers google-generativeai gtts
!pip install -U langchain-community langchain-google-genai
!pip install semantic-kernel
!pip install google-cloud-dialogflow

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from gtts import gTTS
from IPython.display import Audio, display
from google.cloud import dialogflow_v2 as dialogflow
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAITextCompletion
import sqlite3
import uuid
import asyncio
import os


In [ ]:
# Connect to SQLite database (optional if needed)
conn = sqlite3.connect("/content/mahindra_university_data.db")
cursor = conn.cursor()

# (Optional) List tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables:", cursor.fetchall())

In [ ]:
# Sentence Transformers Embedding
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
# Load persisted Chroma vectorstore
vectordb = Chroma(persist_directory="/content/chroma_db", embedding_function=embedding_model)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [ ]:
# Gemini 1.5 Pro model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.7, google_api_key="YOUR_GEMINI_API")


In [ ]:
# Memory for storing conversation
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
# Build RAG Chain with Memory
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)


In [ ]:
# Speak the output
def speak(text, filename="response.mp3"):
    tts = gTTS(text)
    tts.save(filename)
    display(Audio(filename, autoplay=True))


In [ ]:
# Set Google credentials for Dialogflow
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "YOUR_PATH"

def detect_intent_texts(project_id, session_id, texts, language_code="en"):
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(project_id, session_id)

    for text in texts:
        text_input = dialogflow.TextInput(text=text, language_code=language_code)
        query_input = dialogflow.QueryInput(text=text_input)
        response = session_client.detect_intent(request={"session": session, "query_input": query_input})
        return response.query_result.intent.display_name


In [ ]:
!pip install semantic-kernel --upgrade

In [ ]:
!pip install --upgrade semantic-kernel

In [ ]:
import semantic_kernel
print(semantic_kernel.__version__)


In [ ]:
!pip install semantic-kernel --upgrade
!pip install google-generativeai


In [ ]:
import semantic_kernel.connectors.ai as ai_connectors
print(dir(ai_connectors))


In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.google import GoogleAIGeminiChatCompletion
import google.generativeai as genai
import asyncio

# Set Gemini API key
gemini_api_key = "YOUR_API_KEY"
genai.configure(api_key=gemini_api_key)

# Async main
async def main():
    # Initialize the kernel
    kernel = sk.Kernel()

    # Gemini chat completion service
    gemini_service = GoogleAIGeminiChatCompletion(
        model_id="gemini-pro",
        api_key=gemini_api_key,
    )

    # Register the service
    kernel.add_service(gemini_service)

    # Define prompt inline (no file needed)
    prompt = """
    You are a helpful assistant.
    Respond to the following user message:
    {{$input}}
    """

    # Create chat function
    chat_function = kernel.add_function(
        function_name="chat_with_gemini",
        plugin_name="ChatPlugin",
        prompt_template=prompt,
    )

    # Run the chat
    result = await kernel.invoke(chat_function, input="What is intent classification?")
    print("Gemini response:", result)

# If you're in Jupyter/Colab, use this to avoid event loop error
await main()


In [ ]:
# Main Chat Function
def slm_chat(query):
    dialogflow_project_id = "your-dialogflow-project-id"  # change this
    session_id = str(uuid.uuid4())

    # 1. Detect intent
    intent = detect_intent_texts(dialogflow_project_id, session_id, [query])
    print(f"Intent detected: {intent}")

    # 2. Respond based on intent
    if intent == "faq":
        response = rag_chain.invoke({"question": query})["answer"]
    elif intent == "greeting":
        response = asyncio.run(greeting_function.invoke_async(query))
    elif intent == "general_query":
        response = asyncio.run(greeting_function.invoke_async(query))
    else:
        response = "I'm sorry, I didn't understand that."

    print("Bot:", response)
    speak(response)
    return response
